# Intro

## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.distributions.binomial as binomial

from torch.nn.functional import conv2d, max_pool2d, cross_entropy

plt.rc("figure", dpi=100)

## Input data

In [2]:
batch_size = 100

# transform images into normalized tensors
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))
])

train_dataset = datasets.MNIST(
    "./",
    download=True,
    train=True,
    transform=transform,
)

test_dataset = datasets.MNIST(
    "./",
    download=True,
    train=False,
    transform=transform,
)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=1,
    pin_memory=True,
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=1,
    pin_memory=True,
)

## Utility functions

In [3]:
def init_weights(shape):
    # Kaiming He initialization (a good initialization is important)
    # https://arxiv.org/abs/1502.01852
    std = np.sqrt(2. / shape[0])
    w = torch.randn(size=shape) * std
    w.requires_grad = True
    return w


def rectify(x):
    # Rectified Linear Unit (ReLU)
    return torch.max(torch.zeros_like(x), x)


def dropout(X, p_drop):
    m = binomial.Binomial(probs=p_drop)
    
    if not (0 < p_drop < 1):
        return X
    
    for i in range(len(X)):
        if m.sample() == 0:
            X[i] /= (1 - p_drop)
        else:
            X[i].zero_()
            
    return X


class RMSprop(optim.Optimizer):
    """
    This is a reduced version of the PyTorch internal RMSprop optimizer
    It serves here as an example
    """
    def __init__(self, params, lr=1e-3, alpha=0.5, eps=1e-8):
        defaults = dict(lr=lr, alpha=alpha, eps=eps)
        super(RMSprop, self).__init__(params, defaults)

    def step(self):
        for group in self.param_groups:
            for p in group['params']:
                grad = p.grad.data
                state = self.state[p]

                # state initialization
                if len(state) == 0:
                    state['square_avg'] = torch.zeros_like(p.data)

                square_avg = state['square_avg']
                alpha = group['alpha']

                # update running averages
                square_avg.mul_(alpha).addcmul_(grad, grad, value=1 - alpha)
                avg = square_avg.sqrt().add_(group['eps'])

                # gradient update
                p.data.addcdiv_(grad, avg, value=-group['lr'])


# define the neural network
def dropout_model(x, w_h, w_h2, w_o, p_drop_input, p_drop_hidden):
    h = rectify(dropout(x @ w_h, p_drop_input))
    h2 = rectify(dropout(h @ w_h2, p_drop_hidden))
    pre_softmax = h2 @ w_o
    return pre_softmax

## Running the network

Note that the training losses are skewed by dropout (since we're multiplying non-dropped inputs).

In [6]:
fig = plt.gcf()
fig.set_size_inches(15, 5)

for i, p in enumerate([0.1, 0.25, 0.5]):
    print(f"Running model with dropout {p}")
    
    # input shape is (B, 784)
    w_h = init_weights((784, 625))
    # hidden layer with 625 neurons
    w_h2 = init_weights((625, 625))
    # hidden layer with 625 neurons
    w_o = init_weights((625, 10))
    # output shape is (B, 10)

    optimizer = RMSprop(params=[w_h, w_h2, w_o])

    n_epochs = 100

    train_loss = []
    test_loss = []

    # put this into a training loop over 100 epochs
    for epoch in range(n_epochs + 1):
        train_loss_this_epoch = []
        for idx, batch in enumerate(train_dataloader):
            x, y = batch

            # our model requires flattened input
            x = x.reshape(batch_size, 784)
            # feed input through model
            noise_py_x = dropout_model(x, w_h, w_h2, w_o, p_drop_input=p, p_drop_hidden=p)

            # reset the gradient
            optimizer.zero_grad()

            # the cross-entropy loss function already contains the softmax
            loss = cross_entropy(noise_py_x, y, reduction="mean")

            train_loss_this_epoch.append(float(loss))

            # compute the gradient
            loss.backward()

            # update weights
            optimizer.step()

        train_loss.append(np.mean(train_loss_this_epoch))

        # test periodically
        if epoch % 10 == 0:
            print(f"Epoch: {epoch}")
            print(f"Mean Train Loss: {train_loss[-1]:.2e}")
            test_loss_this_epoch = []

            # no need to compute gradients for validation
            with torch.no_grad():
                for idx, batch in enumerate(test_dataloader):
                    x, y = batch
                    x = x.reshape(batch_size, 784)

                    # no dropouts for testing!
                    noise_py_x = dropout_model(x, w_h, w_h2, w_o, p_drop_input=0, p_drop_hidden=0)

                    loss = cross_entropy(noise_py_x, y, reduction="mean")
                    test_loss_this_epoch.append(float(loss))

            test_loss.append(np.mean(test_loss_this_epoch))

            print(f"Mean Test Loss:  {test_loss[-1]:.2e}")

            
    plt.subplot(1, 3, i+1)
    plt.plot(np.arange(n_epochs + 1), train_loss, label=f"Train")
    plt.plot(np.arange(1, n_epochs + 2, 10), test_loss, label="Test")
    plt.title(f"Dropout {p}")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    print()


Running model with dropout 0.1
Epoch: 0
Mean Train Loss: 8.04e-01
Mean Test Loss:  1.56e-01
Epoch: 10
Mean Train Loss: 5.87e-01
Mean Test Loss:  2.52e-01
Epoch: 20
Mean Train Loss: 5.57e-01
Mean Test Loss:  2.80e-01
Epoch: 30
Mean Train Loss: 5.20e-01
Mean Test Loss:  4.59e-01
Epoch: 40
Mean Train Loss: 5.14e-01
Mean Test Loss:  6.84e-01


KeyboardInterrupt: 

<Figure size 1500x500 with 0 Axes>